# Tweet listener (python programming part)

Steps to follow:
* I) Import all necessary libraries to create connection with Twitter, read the tweet and keep it available for streaming.
* II) Read the incoming tweet JSON file (The inflow tweets are in JSON format).
* III) Retrieve only the actual tweet message and sent it to the client socket.
* IV) Define the host and port. Initialized the socket object and bind host and port together.
* V) Establish the connection with Client.
* VI) Use the authentication keys (access_token, access_secret_token, consumer_key and consumer_secret_key) to get the live stream data.
* VII) Filter tweets which contains a specific subjects. In my example I searched tweets related to ‘corona’. We can pass multiple tracking criteria.

In [0]:
# install tweepy into instance
%pip install tweepy

Python interpreter will be restarted.
Collecting tweepy
 Downloading tweepy-3.9.0-py2.py3-none-any.whl (30 kB)
Requirement already satisfied: requests[socks]>=2.11.1 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (2.22.0)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.7/site-packages (from tweepy) (1.14.0)
Collecting requests-oauthlib>=0.7.0
 Downloading requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (2020.6.20)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (2.8)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (1.25.8)
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /usr/lib/python3/dist-packages (from requests[socks]>=2.11.1->tweepy) (3.0.4)
Requirement already satisfied: PySocks!=1.5.7,>=1.5.6; extra == "socks" in /databricks/python3/lib/python3.7/site-packages (from requests[socks]>=2.11.1->tweepy) (1.7.1)
Collecting oauthlib>=3.0.0
 Downloading oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
Installing collected packages: oauthlib, requests-oauthlib, tweepy
Successfully installed oauthlib-3.1.0 requests-oauthlib-1.3.0 tweepy-3.9.0
Python interpreter will be restarted.

In [0]:
# imports
import tweepy 
from tweepy import OAuthHandler # to authenticate Twitter API
from tweepy import Stream 
from tweepy.streaming import StreamListener
import socket 
import json 

In [0]:
# Twitter developer Credentials to connect to twitter account
access_token = "1325893890109476865-WNrvgmwwGYLHVV2EV3vfJjsXxmTVf7"
access_secret = "8dk0wCwqGWTeg8kUssoKlKgig8I9MvaDxw6CpLGFJzgkY"
consumer_key = "C7JV9gkIjwUD3e00TaNGt00Ie"
consumer_secret = "axrsqQv89FrpWpZMX22DcegO7Yse22doz0BVliTyLpEHLPBfAF"


In [0]:
# Tweets listener class
class TweetsListener(StreamListener):
    # initialized the constructor
    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            # read the Twitter data which comes as a JSON format
            msg = json.loads(data)

            # the 'text' in the JSON file contains the actual tweet.
            print(msg['text'].encode('utf-8'))

            # the actual tweet data is sent to the client socket
            self.client_socket.send(msg['text'].encode('utf-8'))
            return True

        except BaseException as e:
            # Error handling
            print("Ahh! Look what is wrong : %s" % str(e))
            return True

    def on_error(self, status):
        print(status)
        return True

In [0]:
def sendData(c_socket):
    # authentication
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # twitter_stream will get the actual live tweet data
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    # filter the tweet feeds related to "saprissa"
    twitter_stream.filter(track=['saprissa'])
    # in case you want to pass multiple criteria
    # twitter_stream.filter(track=['DataScience','python','Iot'])

In [0]:
# create a socket object
s = socket.socket()

# Get local machine name : host and port
host = "127.0.0.1"
port = 3333

# Bind to the port
s.bind((host, port))
print("Listening on port: %s" % str(port))

# Wait and Establish the connection with client.
s.listen(5)
c, addr = s.accept()

print("Received request from: " + str(addr))

# Keep the stream data available
sendData(c)